In [12]:
!pip install groq python-dotenv --quiet

In [13]:
import os
import getpass
from groq import Groq

# Set API Key
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [20]:
# ===============================
# MODEL + EXPERT CONFIGURATION
# ===============================

# ✅ Currently supported Groq model (stable)
MODEL_NAME = "llama-3.3-70b-versatile"

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Senior Technical Support Engineer.
Be precise, logical, and code-focused.
If debugging, explain root cause clearly and provide corrected code.
Keep answers structured and technical."""
    },

    "billing": {
        "system_prompt": """You are a Billing and Subscription Support Specialist.
Be empathetic and professional.
Focus on charges, refunds, subscriptions, invoices, and policy clarity.
Provide next steps clearly."""
    },

    "general": {
        "system_prompt": """You are a helpful customer support assistant.
Answer clearly and conversationally."""
    },

    "tool": {
        "system_prompt": "Tool expert (handled outside LLM)."
    }
}

In [21]:
def route_prompt(user_input):
    routing_prompt = f"""
Classify the following user query into ONE of these categories:
[technical, billing, general, tool]

Return ONLY the category name. No explanation.

Query:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,  # deterministic
        messages=[
            {"role": "system", "content": "You are a strict classifier."},
            {"role": "user", "content": routing_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()
    return category

In [22]:
def get_bitcoin_price():
    # Mock tool response
    return "The current price of Bitcoin is approximately $64,250 (mock data)."

In [23]:
def process_request(user_input):
    category = route_prompt(user_input)
    print(f"\n[Router Decision]: {category}")

    # Tool handling
    if category == "tool":
        return get_bitcoin_price()

    # Fallback safety
    if category not in MODEL_CONFIG:
        category = "general"

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,  # more creative for experts
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

In [24]:
print(process_request("My python script is throwing an IndexError on line 5."))


[Router Decision]: technical
**Error Analysis**

To assist you in resolving the `IndexError` on line 5 of your Python script, I'll need more information. However, I can guide you through a general approach to debugging this issue.

### Possible Causes

An `IndexError` typically occurs when you attempt to access an index in a list or other sequence that does not exist. This can happen for several reasons:

1. **Out-of-range indexing**: The index you're trying to access is greater than or equal to the length of the list.
2. **Empty list**: You're trying to access an element in an empty list.

### Debugging Steps

1. **Verify the line of code**: Check line 5 of your script to see what operation is being performed. Look for list indexing or slicing operations.
2. **Check variable values**: Before the line that's causing the error, print out the values of the variables involved in the indexing operation. This will help you understand what the script is trying to access.
3. **List length**:

In [25]:
print(process_request("I was charged twice for my subscription this month."))


[Router Decision]: billing
I'm so sorry to hear that you were charged twice for your subscription this month. I can understand how frustrating that must be for you.

I'd be happy to help you resolve this issue as quickly as possible. Can you please provide me with more details about the charges? This includes the date of the duplicate charge, the amount, and the method of payment used. Additionally, could you please confirm your subscription details, such as the type of subscription and the billing cycle?

Once I have this information, I can investigate the matter further and work on processing a refund for the duplicate charge. If you have already contacted our team about this issue, please let me know the reference number or the name of the representative you spoke with, so I can pick up where they left off.

Please be assured that we will do our best to rectify the situation and prevent any similar issues in the future. Your satisfaction is our top priority, and I appreciate your p

In [26]:
print(process_request("What is the current price of Bitcoin?"))


[Router Decision]: general
I'd be happy to help you with that. However, please keep in mind that the price of Bitcoin can fluctuate rapidly and may have changed since my knowledge was last updated.

As of my knowledge cutoff, I don't have real-time access to current market prices. But I can suggest some ways for you to find the current price of Bitcoin. You can check a reliable cryptocurrency website or exchange, such as CoinMarketCap or Coinbase, for the most up-to-date pricing information.

If you'd like, I can also guide you through the process of finding the current price or help you with any other Bitcoin-related questions you may have. Just let me know how I can assist you further.


In [27]:
print(process_request("Hi, what services do you offer?"))


[Router Decision]: general
We offer a wide range of services to help with any questions or issues you might be having. Our main goal is to provide you with helpful and accurate information, so you can get back to what's important.

We can assist with things like troubleshooting, answering frequently asked questions, and even help with more complex problems. Whether you're having trouble with a product, need help with an order, or just have a general question, we're here to help.

We also have a vast knowledge base that we can draw from, so even if we don't have the answer right away, we can usually find it for you. And if we can't help with something, we'll do our best to point you in the right direction.

What's on your mind today? Is there something specific you need help with, or would you like me to tell you more about our services?
